In [1]:
import pandas as pd
import os
import numpy as np
import joblib


# Load training data

In [2]:
def loadData(folder_path, X_raw, y_raw):
    
    # get all files in folder
    file_list = [file for file in os.listdir(folder_path) if file.endswith('.xlsx')]
    
    for file in file_list:
        file_path = os.path.join(folder_path, file)
        
        # Read 'stress' as features (X) and 'force' as labels (y)
        X = pd.read_excel(file_path, sheet_name='stress')
        y = pd.read_excel(file_path, sheet_name='force')
        
        X_raw.append(X)
        y_raw.append(y)

folder_path = '../data/train'
X_raw = []
y_raw = []
loadData(folder_path, X_raw, y_raw)

In [3]:
#check dimension for each lab results
for i in range(len(X_raw)):
    print(f"Lab {i + 1:<2} | Feature Shape: {str(X_raw[i].shape):<15} | Label Shape: {str(y_raw[i].shape):<15}")


Lab 1  | Feature Shape: (79679, 19)     | Label Shape: (64512, 4)     
Lab 2  | Feature Shape: (147892, 19)    | Label Shape: (146432, 4)    
Lab 3  | Feature Shape: (106195, 19)    | Label Shape: (84992, 4)     
Lab 4  | Feature Shape: (92476, 19)     | Label Shape: (84992, 4)     


There are different amount of samples for features and label. I wish to make them the same by joining using common feature 'time'

In [4]:
for data in X_raw:
    print(data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 79679 entries, 0 to 79678
Data columns (total 19 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Time      79679 non-null  float64
 1   1_red     79679 non-null  float64
 2   1_blue    79679 non-null  float64
 3   1_yellow  79679 non-null  float64
 4   2_red     79679 non-null  float64
 5   2_blue    79679 non-null  float64
 6   2_yellow  79679 non-null  float64
 7   3_red     79679 non-null  float64
 8   3_blue    79679 non-null  float64
 9   3_yellow  79679 non-null  float64
 10  4_red     79679 non-null  float64
 11  4_blue    79679 non-null  float64
 12  4_yellow  79679 non-null  float64
 13  5_red     79679 non-null  float64
 14  5_blue    79679 non-null  float64
 15  5_yellow  79679 non-null  float64
 16  6_red     79679 non-null  float64
 17  6_blue    79679 non-null  float64
 18  6_yellow  79679 non-null  float64
dtypes: float64(19)
memory usage: 11.6 MB
None
<class 'pandas.core.frame

In [5]:
for data in y_raw:
    print(data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 64512 entries, 0 to 64511
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Time (s)  64512 non-null  float64
 1   Z (kN)    64512 non-null  float64
 2   Y (kN)    64512 non-null  float64
 3   X (kN)    64512 non-null  float64
dtypes: float64(4)
memory usage: 2.0 MB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 146432 entries, 0 to 146431
Data columns (total 4 columns):
 #   Column    Non-Null Count   Dtype  
---  ------    --------------   -----  
 0   Time (s)  146432 non-null  float64
 1   Z (kN)    146432 non-null  float64
 2   Y (kN)    146432 non-null  float64
 3   X (kN)    146432 non-null  float64
dtypes: float64(4)
memory usage: 4.5 MB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 84992 entries, 0 to 84991
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Time (s)  84992 non-null  f

### Data cleaning

In [6]:
# Show full precision of first 5 time values
print(X_raw[0]['Time'].iloc[:5].values)
print(y_raw[0]['Time (s)'].iloc[:5].values)

[0.       0.000488 0.000977 0.001465 0.001953]
[0.00048828 0.00097656 0.00146484 0.00195312 0.00244141]


There are miss match in decimal places for time. 

In [7]:
# clean up time and merge features and labels
def clean_and_merge_features_labels(X, y, time_col_X='Time', time_col_y='Time (s)', rounding_precision=6):
    """
    Cleans and merges stress and force dataframes by inner join on rounded time values.

    Args:
        X (DataFrame): Raw stress data with Time column.
        y (DataFrame): Raw force data with Time column.
        time_col_X (str): Column name for time in X_raw.
        time_col_y (str): Column name for time in y_raw.
        rounding_precision (int): Decimal places to round time values to.

    Returns:
        X_aligned (DataFrame): Cleaned and aligned feature set (including Time if needed).
        y_aligned (DataFrame): Cleaned and aligned label set (X, Y, Z forces).
    """
    # Round time columns to ensure alignment
    X[time_col_X] = X[time_col_X].round(rounding_precision)
    y[time_col_y] = y[time_col_y].round(rounding_precision)
    
    # Merge on rounded time
    merged = pd.merge(X, y, left_on=time_col_X, right_on=time_col_y, how='inner')
    
    # Keep only one time column 
    merged = merged.drop(columns=[time_col_y])

    # Separate features and labels 
    X_aligned = merged.iloc[:, :-3]  # Features include Time and stress sensors
    y_aligned = merged.iloc[:, -3:]  # Labels: X, Y, Z forces

    return X_aligned, y_aligned


In [8]:
X_aligned_list = []
y_aligned_list = []

for X, y in zip(X_raw, y_raw):
    X_aligned, y_aligned = clean_and_merge_features_labels(X, y)
    X_aligned_list.append(X_aligned)
    y_aligned_list.append(y_aligned)

In [9]:
for i in range(len(X_aligned_list)):
    print(f"Lab {i + 1}")
    print(f"\tDimension X: ", X_aligned_list[i].shape)
    print(f"\tDimension Y: ", y_aligned_list[i].shape)
    print(f"\tMissing values:", (X_aligned_list[i].isna().sum()).sum())
    print(f"\tDuplicates: ", X_aligned_list[i].duplicated().sum())
    print("-----------------------------------------------")

Lab 1
	Dimension X:  (63504, 19)
	Dimension Y:  (63504, 3)
	Missing values: 0
	Duplicates:  0
-----------------------------------------------
Lab 2
	Dimension X:  (144144, 19)
	Dimension Y:  (144144, 3)
	Missing values: 0
	Duplicates:  0
-----------------------------------------------
Lab 3
	Dimension X:  (83664, 19)
	Dimension Y:  (83664, 3)
	Missing values: 0
	Duplicates:  0
-----------------------------------------------
Lab 4
	Dimension X:  (83664, 19)
	Dimension Y:  (83664, 3)
	Missing values: 0
	Duplicates:  0
-----------------------------------------------


In [10]:
for i, y in enumerate(y_aligned_list):
    print(f"lab {i + 1}")
    print(f"y initial: ", y_raw[i].shape)
    print(f"y after join: ", y_aligned_list[i].shape)
    print("-" * 20)
    

lab 1
y initial:  (64512, 4)
y after join:  (63504, 3)
--------------------
lab 2
y initial:  (146432, 4)
y after join:  (144144, 3)
--------------------
lab 3
y initial:  (84992, 4)
y after join:  (83664, 3)
--------------------
lab 4
y initial:  (84992, 4)
y after join:  (83664, 3)
--------------------


In [11]:
y_aligned_list[0].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 63504 entries, 0 to 63503
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Z (kN)  63504 non-null  float64
 1   Y (kN)  63504 non-null  float64
 2   X (kN)  63504 non-null  float64
dtypes: float64(3)
memory usage: 1.9 MB


In [12]:
### save df 
joblib.dump(X_aligned_list, '../data/interim/01_X_aligned_list.joblib')
joblib.dump(y_aligned_list, '../data/interim/02_y_aligned_list.joblib')

['../data/interim/02_y_aligned_list.joblib']